In [1]:
import tensorflow as tf 

In [2]:
tf.__version__

'2.3.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [4]:
train_images = tf.expand_dims(train_images, -1)
test_images = tf.expand_dims(test_images, -1)

In [5]:
train_images = tf.cast(train_images/255, tf.float32)
test_images = tf.cast(test_images/255, tf.float32)

In [6]:
train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [8]:
dataset = dataset.shuffle(10000).batch(32)
test_dataset = test_dataset.batch(32)

In [9]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [11]:
optimizer = tf.keras.optimizers.Adam()

In [12]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [13]:
features, labels = next(iter(dataset))

In [14]:
features.shape

TensorShape([32, 28, 28, 1])

In [15]:
labels.shape

TensorShape([32])

In [16]:
predictions = model(features)

In [17]:
predictions.shape

TensorShape([32, 10])

In [18]:
tf.argmax(predictions, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3, 3, 0, 0, 3, 3, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3,
       0, 3, 0, 3, 0, 3, 3, 3, 3, 0], dtype=int64)>

In [19]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([5, 7, 4, 4, 4, 7, 8, 9, 5, 5, 5, 5, 5, 8, 9, 6, 4, 6, 3, 3, 0, 9,
       0, 8, 1, 4, 6, 6, 7, 3, 6, 6], dtype=int64)>

In [20]:
def cal_loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [21]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [22]:
def train_step(model, optimizer, images, labels):
    with tf.GradientTape() as tape:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = tape.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [23]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [24]:
def train(epochs, model, optimizer, dataset):
    for epoch in range(epochs):
        for (batch_index, (images, labels)) in enumerate(dataset):
            train_step(model, optimizer, images, labels)
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch, train_loss.result(), train_accuracy.result()))
        
        for (batch_index, (images, labels)) in enumerate(test_dataset):
            test_step(model, optimizer, images, labels)
        print('Epoch{} test_loss is {}, test_accuracy is {}'.format(epoch, test_loss.result(), test_accuracy.result()))
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [25]:
# train(10, model, optimizer, dataset)